In [1]:
import pandas as pd
import zipfile
from io import BytesIO
import requests
import json


In [ ]:
# Lendo direto assim não rolou
json = pd.read_json('https://sistemaswebb3-listados.b3.com.br/investorParticipationProxy/investorParticipationCall/GetOpendaily/eyJwYWdlU2l6ZSI6MjAsImlkZW50aWZpZXIiOm51bGwsImxhbmd1YWdlIjoicHQtYnIifQ==')
json

In [2]:
# Url pega no inspecionando o elemento e networdk > tinha um get que o Doug achou e era esse url
url = 'https://sistemaswebb3-listados.b3.com.br/investorParticipationProxy/investorParticipationCall/GetOpendaily/eyJwYWdlU2l6ZSI6MjAsImlkZW50aWZpZXIiOm51bGwsImxhbmd1YWdlIjoicHQtYnIifQ=='

r = requests.get(url)
r

<Response [200]>

In [3]:
# Opa, deu bom
r.content

b'{"HeaderDates":{"StartDate":"2022-09-01T00:00:00","EndDate":"2022-09-22T00:00:00","UpdateDate":"0001-01-01T00:00:00"},"HeaderDaily":{"TypeInvestorTitle":"Tipo de Investidores","BuyVolumeTitle":"Vol_Compra_R$ MIL","BuyParticipationTitle":"Part_Compra (%)","SaleVolumeTitle":"Vol_Venda_R$ MIL","SaleParticipationTitle":"Part_Venda (%)"},"Result":[{"TypeInvestor":"Investidores Individuais","BuyVolume":"64.962.887","BuyParticipation":"7,47%","SaleVolume":"63.999.172","SaleParticipation":"7,36%"},{"TypeInvestor":"Institucionais","BuyVolume":"107.690.834","BuyParticipation":"12,39%","SaleVolume":"111.074.808","SaleParticipation":"12,78%"},{"TypeInvestor":"Investidor Estrangeiro","BuyVolume":"242.099.653","BuyParticipation":"27,85%","SaleVolume":"243.068.194","SaleParticipation":"27,96%"},{"TypeInvestor":"Institui\xc3\xa7\xc3\xb5es Financeiras","BuyVolume":"14.399.644","BuyParticipation":"1,66%","SaleVolume":"14.333.854","SaleParticipation":"1,65%"},{"TypeInvestor":"Outros","BuyVolume":"5.519

##### Convertendo o json em pandasdataframe

In [8]:
# Aqui não expande o result - mas ajuda para pegar a data
import json
data = pd.json_normalize(json.loads(r.content))
data

,Result,HeaderDates.StartDate,HeaderDates.EndDate,HeaderDates.UpdateDate,HeaderDaily.TypeInvestorTitle,HeaderDaily.BuyVolumeTitle,HeaderDaily.BuyParticipationTitle,HeaderDaily.SaleVolumeTitle,HeaderDaily.SaleParticipationTitle
0,"[{'TypeInvestor': 'Investidores Individuais', ...",2022-09-01T00:00:00,2022-09-22T00:00:00,0001-01-01T00:00:00,Tipo de Investidores,Vol_Compra_R$ MIL,Part_Compra (%),Vol_Venda_R$ MIL,Part_Venda (%)


In [20]:
data_final = pd.to_datetime(data['HeaderDates.EndDate'][0]).strftime('%Y-%m-%d')
data_final

'2022-09-22'

In [5]:
# Expandindo o result com base no link:
# # https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8


final =  pd.json_normalize(json.loads(r.content), record_path =['Result'])
final

,TypeInvestor,BuyVolume,BuyParticipation,SaleVolume,SaleParticipation
0,Investidores Individuais,64.962.887,"7,47%",63.999.172,"7,36%"
1,Institucionais,107.690.834,"12,39%",111.074.808,"12,78%"
2,Investidor Estrangeiro,242.099.653,"27,85%",243.068.194,"27,96%"
3,Instituições Financeiras,14.399.644,"1,66%",14.333.854,"1,65%"
4,Outros,5.519.914,"0,63%",2.196.906,"0,25%"


In [ ]:
final.to_excel('teste.xlsx', index=False)

#### Testando com outro dia
